# Holiday Modeling Using Google Trends Data

### Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import matplotlib

import cmdstanpy
cmdstanpy.install_cmdstan()
from cmdstanpy import CmdStanModel

from bayesian_holidays.src.holiday_model import fit_holiday_model
from bayesian_holidays.src.plot_utils import (
    plot_posteriors,
    get_individual_holidays,
    plot_individual_holidays
)

Installing CmdStan version: 2.30.1
Install directory: /Users/daniel.marthaler/.cmdstan
CmdStan version 2.30.1 already installed


### Choose data and fit model

In [ ]:
df_xmas, fit_xmas = fit_holiday_model(
    "chocolate",
    start_date=None,
    train_split=80,
    num_chains=2,
    max_treedepth=15,
    adapt_delta=0.99
)


16:19:36 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

### Plot posterior in hold out set

In [ ]:
plot_posteriors(df_xmas, fit_xmas,"chocolate")

### Plot Holiday Components for this search term

In [ ]:
holiday_list, hols_train, hols_test, df_train, df_test = get_individual_holidays(df_xmas, fit_xmas)

In [ ]:
plot_individual_holidays(df.date,np.concatenate((hols_train, hols_test), axis=2), holiday_list.head(hols_test.shape[1]).HolidayName.values)

(500, 12, 260)